# Implementing RAG with Pinecone

https://github.com/openai/openai-cookbook/blob/main/examples/vector_databases/pinecone/Using_Pinecone_for_embeddings_search.ipynb

## Creating Vector Database in Pinecone
Important: This part should be only run once. It is the creation of the index in Pinecone.

### Loading the data

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import ast
from typing import List, Iterator

# Function to set the wd as the root of the repository
def find_repo_root(repo_name):
    current_dir = os.getcwd()
    while current_dir != '/':
        if os.path.basename(current_dir) == repo_name:
            return current_dir
        current_dir = os.path.dirname(current_dir)
    raise FileNotFoundError(f"Repository root '{repo_name}' not found.")

# Setting the working directory
repo_name = 'orare-model'
repo_root = find_repo_root(repo_name)
os.chdir(repo_root)

In [2]:
# Reading the bible interpreted with embeddings
bible_data = pd.read_csv('bible/data/bible_by_theme_int_embedding.txt', sep='|', encoding='utf-8')

# Tranforming vector string into list
bible_data['interpretacion_vector'] = bible_data['interpretacion_vector'].apply(ast.literal_eval)

# Showing the bible_data object
bible_data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     861 non-null    object
 1   pasaje                 861 non-null    object
 2   texto                  861 non-null    object
 3   interpretación         861 non-null    object
 4   temas                  861 non-null    object
 5   área_vida              861 non-null    object
 6   texto_vector           861 non-null    object
 7   interpretacion_vector  861 non-null    object
dtypes: object(8)
memory usage: 53.9+ KB


In [3]:
# Formatting the Bible dataframe
# Transforming index as numeric value
bible_data['id'] = bible_data['id'].str.replace('vec', '').astype(int)

# Craeting column "metadata"
bible_data['metadata'] = bible_data.apply(lambda row: {
    'pasaje': row['pasaje'],
    'texto': row['texto'],
    'interpretacion': row['interpretación'],
    'temas': row['temas'],
    'area_vida': row['área_vida']
}, axis=1)

# Dropping the columns inserted in metadata
bible_data = bible_data.drop(columns=['pasaje','texto','interpretación','temas','área_vida'])

# Reordering columns
columns = ['id','interpretacion_vector','metadata']
bible_data = bible_data[columns]
bible_data.head()

,id,interpretacion_vector,metadata
0,1,"[0.05086807906627655, 0.006340509746223688, 0....","{'pasaje': '1 Corintios 10:12', 'texto': 'Así ..."
1,2,"[0.04511460289359093, -0.013647807762026787, -...","{'pasaje': '1 Corintios 10:13', 'texto': 'No o..."
2,3,"[0.0449271984398365, 0.04290250688791275, -0.0...","{'pasaje': '1 Corintios 10:31', 'texto': 'Si p..."
3,4,"[0.06745994836091995, 0.06209307909011841, -0....","{'pasaje': '1 Corintios 11:9', 'texto': 'Porqu..."
4,5,"[0.030204113572835922, -0.004405524581670761, ...","{'pasaje': '1 Corintios 13:13', 'texto': 'Y ah..."


### Setting the connection to OpenAI API and Pinecone API

In [4]:
import openai
import os
import pinecone
from pinecone import Pinecone

# Pinecone API
pc_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pc_api_key)

# OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

/Users/hiroshi/Documents/ds_projects/orare-model/venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Helper function

In [11]:
# Models a simple batch generator that make chunks out of an input DataFrame
class BatchGenerator:
   
    def __init__(self, batch_size: int = 10) -> None:
        self.batch_size = batch_size
    
    # Makes chunks out of an input DataFrame
    def to_batches(self, df: pd.DataFrame) -> Iterator[pd.DataFrame]:
        splits = self.splits_num(df.shape[0])
        if splits <= 1:
            yield df
        else:
            for chunk in np.array_split(df, splits):
                yield chunk

    # Determines how many chunks DataFrame contains
    def splits_num(self, elements: int) -> int:
        return int(np.ceil(elements / self.batch_size))
    
    __call__ = to_batches

df_batcher = BatchGenerator(300)

### Creating an Index

In [48]:
# Create the Index in the Pinecone webpage

### Loading the index

In [ ]:
index_name = 'bible-verses-metadata'

# Setting the host of the index from the Pinecone web admin portal
index = pc.Index(index_name=index_name, host='https://bible-verses-metadata-rsup9mo.svc.aped-4627-b74a.pinecone.io')

### Uploading the vectors to the content namespace

In [13]:
# Upsert content vectors in content namespace - this can take a few minutes
print("Uploading vectors to content namespace..")
for batch_df in df_batcher(bible_data):
    vectors = list(zip(batch_df.id.astype(str), batch_df.interpretacion_vector, batch_df.metadata))
    index.upsert(vectors=vectors, namespace='content')

Uploading vectors to content namespace..


In [25]:
# Check index size of the namespace to confirm all of our docs have been loaded
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'content': {'vector_count': 861}},
 'total_vector_count': 861}

## Vector Search Query

In [6]:
# Importing the libraries and setting the clients
import pandas as pd
import openai
import os
# Pinecone's client library for Python
import pinecone
from pinecone import Pinecone

# Pinecone API
pc_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pc_api_key)

# OpenAI API
openai.api_key = os.getenv('OPENAI_API_KEY')
client = openai.OpenAI()

### Loading Index

In [7]:
index_name = 'bible-verses-metadata'

# Setting the host of the index from the Pinecone web admin portal
index = pc.Index(index_name=index_name, host='https://bible-verses-metadata-rsup9mo.svc.aped-4627-b74a.pinecone.io')

### Helper Functions

In [19]:
# Function to flatten the text in a single line
def flatten_text(text):
    flattened_text = text.replace('\n', ' ')
    flattened_text = ' '.join(flattened_text.split())
    return flattened_text

# Function to insert line breaks
def insert_line_breaks(text, limit):
    lines = []
    while len(text) > limit:
        breakpoint = text.rfind(' ', 0, limit)
        if breakpoint == -1:  # No space found, force break
            breakpoint = limit
        lines.append(text[:breakpoint])
        text = text[breakpoint:].lstrip()  # Remove leading spaces from the remaining text
    lines.append(text)
    return '\n'.join(lines)

# Function to separate the paragraphs before insert line breaks
def format_text_with_line_breaks(text, limit):
    paragraphs = text.split('\n\n')
    formatted_paragraphs = [insert_line_breaks(paragraph, limit) for paragraph in paragraphs]
    return '\n\n'.join(formatted_paragraphs)

# Function to get the embeddings of the input of the search
def get_embedding(text, model="text-embedding-3-small"):
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

# Function to process a prompt in ChatGPT
def get_completion_4o(prompt, model="gpt-4o", temperature=1, top_p=1):
    """
    Prompt completion function

    Parameters:
    prompt (str): The prompt to be completed
    model (str): The model to be used for completion.
    temperature (float): The temperature to be used for completion.
                        range: [0, 2]
                        Lower values (towards 0) make the model more deterministic.
                        Higher values (towards 2) make the model more random.
    top_p (float): The top_p to be used for completion.
                        range: [0, 1]
                        Lower values make the model more deterministic.
                        Higher values make the model more random.
    """
    messages=[
       {"role": "system", "content": "Tú eres un sacerdote de una iglesia católica"},
       {"role": "user", "content": prompt}
       ]
    #messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p
    )
    return response.choices[0].message.content

### Verse Recommender Function

In [17]:
# Function to process the output of the Vector search and generate a prompt for ChatGPT to generate the final output for the user
def verse_recommender(prayer, temperature=1, top_p=1):
    # Flattening the prayer
    prayer = flatten_text(prayer)
    # Embedding the prayer
    embedded_prayer = get_embedding(prayer)
    # Vector database search
    query_result = index.query(vector=embedded_prayer, 
                               top_k=3,
                               namespace='content',
                               include_metadata=True)
    
    # Splitting the results of the search
    score1 = query_result['matches'][0]['score']
    score2 = query_result['matches'][1]['score']
    score3 = query_result['matches'][2]['score']
    passage1 = query_result['matches'][0]['metadata']['pasaje']
    passage2 = query_result['matches'][1]['metadata']['pasaje']
    passage3 = query_result['matches'][2]['metadata']['pasaje']
    text1 = query_result['matches'][0]['metadata']['texto']
    text2 = query_result['matches'][1]['metadata']['texto']
    text3 = query_result['matches'][2]['metadata']['texto']
    inter1 = query_result['matches'][0]['metadata']['interpretacion']
    inter2 = query_result['matches'][1]['metadata']['interpretacion']
    inter3 = query_result['matches'][2]['metadata']['interpretacion']

    
    # Prompt with Bible's verses resulted from the vector search
    # In this Prompt we take as input the "prayer" and the "Bible's verses recommened"
    # The ouptut is the interpretation of the "Bible's verse recommened" according to the prayer and some words of encouragement
    prompt = f"""
                # Rol #
                Tu eres el GPT: Catholic Bible Guide by Fr. Abraham Mutholath.
                Tu rol será el de un sacerdote de una iglesia Católica. Se te dará una oración de una persona y 3 pasajes de la Biblia relacionados a la oración. 
                Tu tarea será elegir el pasaje de la Biblia que más relación tenga con la oración de la persona. Luego deberás interpretar el pasaje de la Biblia que elegiste acorde a la oración de la persona y dar unas palabras de aliento.
                # Fin Rol #

                # Tareas #
                - Leer la oración de la persona y los 3 pasajes de la Biblia.
                - Elegir el pasaje de la Biblia que más relación tenga con la oración de la persona.
                - Interpretar el pasaje de la Biblia acorde a la oración de la persona y dar unas palabras de aliento.
                # fin Tareas #

                # Estructura input #
                - Oración de la persona
                - Pasaje de la Biblia 1
                - Pasaje de la Biblia 2
                - Pasaje de la Biblia 3
                # Fin estructura input #

                # Input #
                - Oración: {prayer}
                - Pasaje de la Biblia 1: {passage1} :  {text1}
                - Pasaje de la Biblia 2: {passage2} :  {text2}
                - Pasaje de la Biblia 3: {passage3} :  {text3}
                # fin Input #

                # Estructura Output #
                versiculo elegido
                Párrafo con la interpretación del pasaje de la Biblia y palabras de aliento
                # Fin Estructura Output #

                # Ejemplo Output #
                Salmo 22:24 Porque no menospreció ni abominó la aflicción del pobre, Ni de él escondió su rostro; Sino que cuando clamó á él, oyóle.

                Este Salmo nos recuerda la infinita misericordia y compasión de Dios hacia los pobres y afligidos. Este versículo nos asegura que Dios no ignora el sufrimiento de los necesitados, ni les da la espalda. Al contrario, Él escucha sus clamores y está presente en sus momentos de angustia.
                Tu oración por los pobres del mundo es un acto de amor y solidaridad que refleja el corazón de Dios. Al interceder por ellos, te unes a la misión de Cristo de traer consuelo y esperanza a los más vulnerables. Recuerda que Dios escucha nuestras oraciones y actúa a través de nosotros para llevar su amor y provisión a aquellos que más lo necesitan.
                Te animo a seguir orando y, si es posible, a tomar acciones concretas para ayudar a los pobres en tu comunidad. Cada pequeño gesto de generosidad y compasión puede ser una manifestación del amor de Dios en sus vidas. Confía en que Dios, en su infinita bondad, no abandonará a los necesitados y usará nuestras oraciones y acciones para bendecirlos.
                Que el Señor te bendiga y te fortalezca en tu deseo de servir a los demás. Amén.
                # Fin Ejemplo Ouput #
              """
    # Function to process the prompt
    response_4o = get_completion_4o(prompt, temperature=temperature, top_p=top_p)
    # Formatting the answer of ChatGPT
    response_4o = response_4o.lstrip()
    num_char_linebreak = 130
    response_4o_break = format_text_with_line_breaks(response_4o, num_char_linebreak)
    # Formatting the prayer
    prayer_break = format_text_with_line_breaks(prayer, num_char_linebreak)
    # Formatting the Bible verses and its interpretation from the Vector Search
    passage1 = format_text_with_line_breaks(passage1,num_char_linebreak)
    passage2 = format_text_with_line_breaks(passage2,num_char_linebreak)
    passage3 = format_text_with_line_breaks(passage3,num_char_linebreak)
    text1 = format_text_with_line_breaks(text1,num_char_linebreak)
    text2 = format_text_with_line_breaks(text2,num_char_linebreak)
    text3 = format_text_with_line_breaks(text3,num_char_linebreak)
    inter1 = format_text_with_line_breaks(inter1,num_char_linebreak)
    inter2 = format_text_with_line_breaks(inter2,num_char_linebreak)
    inter3 = format_text_with_line_breaks(inter3,num_char_linebreak)
    # Printing all the verses recommened in order to check the process (For developer's use only)
    print('Pasajes recomendados')
    print('---------------------------')
    print(f"- Score: {score1}")
    print(f"- {passage1}: {text1}")
    print(f"- Interpretación: {inter1}")
    print('---------------------------')
    print(f"- Score: {score2}")
    print(f"- {passage2}: {text2}")
    print(f"- Interpretación: {inter2}")
    print('---------------------------')
    print(f"- Score: {score3}")
    print(f"- {passage3}: {text3}")
    print(f"- Interpretación: {inter3}")
    # Printing the final output of the chat
    print('---------------------------')
    print('Output Orare')
    print('---------------------------')
    print(f'Oración: {prayer_break}')
    print('---------------------------')
    print(response_4o_break)

    return

https://community.openai.com/t/cheat-sheet-mastering-temperature-and-top-p-in-chatgpt-api/172683

According to this blog for chat generation they recommend the following paramenters.


| Parameter | Range | Explanation | Recommended | Default |
| ---- | ---- | ---- | ---- | ---- |
| temperature | [0, 2] | Lower values (towards 0) make the model more deterministic. Higher values (towards 2) make the model more random. | 0.5 | 1 |
| top_p | [0, 1] | Lower values make the model more deterministic. Higher values make the model more random. | 0.5 | 1 |

In [20]:
prayer = "Dios, te pido y oro por los pobres del mundo que no tienen que comer. Por favor se misericordioso con ellos. Pido por ellos. Amen"
verse_recommender(prayer, temperature=1, top_p=1)

Pasajes recomendados
---------------------------
- Score: 0.527822
- Malaquías 3:10: Traed todos los diezmos al alfolí, y haya alimento en mi casa; y probadme ahora en esto, dice Jehová de los ejércitos, si no os
abriré las ventanas de los cielos, y vaciaré sobre vosotros bendición hasta que sobreabunde.
- Interpretación: Dios está pidiendo a su pueblo que traigan los diezmos (una décima parte de sus ingresos) al templo para que haya suficiente
provisión. Él promete que si obedecen, abrirá las ventanas del cielo y derramará bendiciones abundantes sobre ellos.
---------------------------
- Score: 0.522710145
- Deuteronomio 15:11: Porque no faltarán menesterosos de en medio de la tierra; por eso yo te mando, diciendo: Abrirás tu mano á tu hermano, á tu
pobre, y á tu menesteroso en tu tierra.
- Interpretación: Este versículo nos recuerda que siempre habrá personas necesitadas entre nosotros. Por lo tanto, Dios nos manda ser generosos y
ayudar a nuestros hermanos, especialmente a los pobre

In [18]:
prayer = "Dios, te pido y oro por los pobres del mundo que no tienen que comer. Por favor se misericordioso con ellos. Pido por ellos. Amen"
verse_recommender(prayer, temperature=0.5, top_p=0.5)

Pasajes recomendados
---------------------------
- Score: 0.527822
- Malaquías 3:10: Traed todos los diezmos al alfolí, y haya alimento en mi casa; y probadme ahora en esto, dice Jehová de los ejércitos, si no os
abriré las ventanas de los cielos, y vaciaré sobre vosotros bendición hasta que sobreabunde.
- Interpretación: Dios está pidiendo a su pueblo que traigan los diezmos (una décima parte de sus ingresos) al templo para que haya suficiente
provisión. Él promete que si obedecen, abrirá las ventanas del cielo y derramará bendiciones abundantes sobre ellos.
---------------------------
- Score: 0.522710145
- Deuteronomio 15:11: Porque no faltarán menesterosos de en medio de la tierra; por eso yo te mando, diciendo: Abrirás tu mano á tu hermano, á tu
pobre, y á tu menesteroso en tu tierra.
- Interpretación: Este versículo nos recuerda que siempre habrá personas necesitadas entre nosotros. Por lo tanto, Dios nos manda ser generosos y
ayudar a nuestros hermanos, especialmente a los pobre

In [24]:
prayer = '''Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti.
¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, 
pero entiendo espiritualmente lo que estás haciendo conmigo.
Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, 
entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú.
No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, 
respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.
Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, 
aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, 
que su angel guardián le cuente que estamos todos pendientes de él, 
que no contamos los minutos para tenerlo en nuestros brazos, pero que sea paciente,
que tú tienes varios corazones que limpiar en estos momentos, que su venida está generando santidad en nuestro alrededor.'''
verse_recommender(prayer, temperature=1, top_p=1)

Pasajes recomendados
---------------------------
- Score: 0.544656634
- 2 Colosenses 3:5: Y el Señor enderece vuestros corazones en el amor de Dios, y en la paciencia de Cristo.
- Interpretación: Este versículo es una oración para que el Señor guíe y fortalezca los corazones de los creyentes en el amor de Dios y en la
paciencia que Cristo mostró. Se trata de una petición para que los fieles puedan vivir de acuerdo con estos valores.
---------------------------
- Score: 0.534210742
- 2 Samuel 22:5-7: Cuando me cercaron ondas de muerte, Y arroyos de iniquidad me asombraron, Me rodearon los dolores del infierno, Y me tomaron
descuidado lazos de muerte. Tuve angustia, invoqué á Jehová, Y clamé á mi Dios: Y él oyó mi voz desde su templo; Llegó mi clamor
á sus oídos.
- Interpretación: El fragmento describe una situación de gran angustia y peligro, donde el autor se siente rodeado por la muerte y el mal. En su
desesperación, clama a Dios, quien escucha su súplica y responde. Este pasaje resal

In [23]:
prayer = '''Ilumíname señor, dame paz en estos momentos de incertidumbre, sujétame de tu mano, permíteme caminar junto a ti.
¿Por qué permites este dolor dentro de nosotros? ¿Por qué este sufrimiento tan profundo? No lograré entender tus motivos, 
pero entiendo espiritualmente lo que estás haciendo conmigo.
Purificas mi alma, me permitiste soltar el control, afianzarme a ti, volver a mirarte cara a cara, 
entender que todo depende de ti, reconocer la vanidad de mis actos, sentir miedo a cada instante y que mi única fuerza seas tú.
No entiendo tus caminos pero decido caminar dentro de ellos, seguir tu luz, guiar a mi familia hacia tus designios, 
respirar en esa absoluta incertidumbre y orar con todas mis fuerzas para que se haga según tu voluntad.
Cuida a mi familia señor, bendice a mi esposa, cúbrela con tu manto sagrado, fortalece a mi hijo, 
aliéntalo con tu espíritu, dale el soporte que hoy nosotros no podemos darle, 
que su angel guardián le cuente que estamos todos pendientes de él, 
que no contamos los minutos para tenerlo en nuestros brazos, pero que sea paciente,
que tú tienes varios corazones que limpiar en estos momentos, que su venida está generando santidad en nuestro alrededor.'''
verse_recommender(prayer, temperature=0.5, top_p=0.5)

Pasajes recomendados
---------------------------
- Score: 0.544656634
- 2 Colosenses 3:5: Y el Señor enderece vuestros corazones en el amor de Dios, y en la paciencia de Cristo.
- Interpretación: Este versículo es una oración para que el Señor guíe y fortalezca los corazones de los creyentes en el amor de Dios y en la
paciencia que Cristo mostró. Se trata de una petición para que los fieles puedan vivir de acuerdo con estos valores.
---------------------------
- Score: 0.534210742
- 2 Samuel 22:5-7: Cuando me cercaron ondas de muerte, Y arroyos de iniquidad me asombraron, Me rodearon los dolores del infierno, Y me tomaron
descuidado lazos de muerte. Tuve angustia, invoqué á Jehová, Y clamé á mi Dios: Y él oyó mi voz desde su templo; Llegó mi clamor
á sus oídos.
- Interpretación: El fragmento describe una situación de gran angustia y peligro, donde el autor se siente rodeado por la muerte y el mal. En su
desesperación, clama a Dios, quien escucha su súplica y responde. Este pasaje resal